In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import cv2
import matplotlib.pyplot as plt

Fontconfig warning: ignoring UTF-8: not a valid region tag


In [10]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras import layers, models

# Initialize MobileNet pre-trained on ImageNet
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers in the base model
base_model.trainable = False

# Build the custom classification model
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)  # Keep MobileNet in inference mode
x = layers.GlobalAveragePooling2D()(x)  # Convert feature maps to a single vector
x = layers.Dense(128, activation='relu')(x)  # Fully connected layer
x = layers.Dropout(0.5)(x)  # Reduce overfitting
outputs = layers.Dense(8, activation='softmax')(x)  # 8 output classes

# Define model
model = models.Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d_1   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 128)               131200    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 8)                 1032  

In [11]:
import os
import cv2
import numpy as np

def load_images_and_labels(directory, img_size=(224, 224)):
    """
    Load images and labels from a dataset directory.

    Args:
        directory (str): Path to the dataset directory (train, test, val).
        img_size (tuple): Target size for image resizing (default: (224, 224)).

    Returns:
        np.ndarray: Preprocessed images.
        np.ndarray: Corresponding labels.
    """
    images = []
    labels = []
    class_names = os.listdir(directory)
    class_mapping = {name: idx for idx, name in enumerate(class_names)}

    for class_name in class_names:
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):  # Ensure it's a directory
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, img_size)
                    images.append(img)
                    labels.append(class_mapping[class_name])

    images = np.array(images, dtype=np.float32) / 255.0  # Normalize images
    labels = np.array(labels, dtype=np.int32)
    return images, labels

In [12]:
# Define paths to datasets
train_path = '/Users/shubhampund9767/Documents/Cotton-Disease-Recognition-using-YOLO-Algorithm--1/Cotton-Disease-Specific/trainning/images'
test_path = '/content/drive/MyDrive/Cotton Disease/test'
val_path = '/Users/shubhampund9767/Documents/Cotton-Disease-Recognition-using-YOLO-Algorithm--1/Cotton-Disease-Specific/validation/images'

# Load datasets
X_train, y_train = load_images_and_labels(train_path)

X_val, y_val = load_images_and_labels(val_path)

print(f"Training data: {len(X_train)} samples")

print(f"Validation data: {len(X_val)} samples")

Training data: 319 samples
Validation data: 357 samples


In [13]:
from sklearn.preprocessing import LabelBinarizer

# One-hot encode labels
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)

y_val = lb.transform(y_val)

print(f"Training labels shape: {y_train.shape}")

print(f"Validation labels shape: {y_val.shape}")

Training labels shape: (319, 8)
Validation labels shape: (357, 8)


In [14]:
import tensorflow as tf

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(32)


In [31]:
# Train the model
history = model.fit(
    train_dataset,
    epochs=50,  # Adjust epochs as needed
    validation_data=val_dataset
)

Epoch 1/50
10/10 [==============================] - 5s 493ms/step - loss: 0.1141 - accuracy: 0.9749 - val_loss: 7.3881 - val_accuracy: 0.1092
Epoch 2/50
10/10 [==============================] - 5s 482ms/step - loss: 0.1141 - accuracy: 0.9749 - val_loss: 7.6354 - val_accuracy: 0.1092
Epoch 3/50
10/10 [==============================] - 5s 489ms/step - loss: 0.1061 - accuracy: 0.9812 - val_loss: 7.6546 - val_accuracy: 0.1092
Epoch 4/50
10/10 [==============================] - 4s 457ms/step - loss: 0.0804 - accuracy: 0.9843 - val_loss: 7.8139 - val_accuracy: 0.1092
Epoch 5/50
10/10 [==============================] - 5s 487ms/step - loss: 0.0631 - accuracy: 0.9937 - val_loss: 8.2341 - val_accuracy: 0.1092
Epoch 6/50
10/10 [==============================] - 4s 464ms/step - loss: 0.0829 - accuracy: 0.9843 - val_loss: 8.3822 - val_accuracy: 0.1092
Epoch 7/50
10/10 [==============================] - 5s 490ms/step - loss: 0.0554 - accuracy: 0.9937 - val_loss: 8.5133 - val_accuracy: 0.1092
Epoch 

In [32]:
model.save("new_cotton_disease_model.h5")  # Saves as a directory with all required files

In [33]:
loaded_model = tf.keras.models.load_model("/Users/shubhampund9767/Documents/Cotton-Disease-Recognition-using-YOLO-Algorithm--1/new_cotton_disease_model.h5")

In [34]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
def predict_image(image_path, class_names):
    """
    Predicts the class of an image using the trained model.

    Args:
        image_path (str): Path to the image to predict.
        class_names (list): List of class labels.

    Returns:
        str: Predicted class label.
        float: Confidence score of the prediction.
    """
    # Load and preprocess the image
    img = load_img(image_path, target_size=(224, 224))  # Replace with your model's input size
    img_array = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Make prediction
    predictions = model.predict(img_array)
    print(f"Raw predictions: {predictions}")  # Debugging output

    # Get the predicted class index and confidence
    predicted_index = np.argmax(predictions[0])
    confidence = predictions[0][predicted_index]

    # Check if predicted index is valid
    if predicted_index >= len(class_names):
        raise ValueError(
            f"Predicted index {predicted_index} exceeds the number of class labels in class_names {len(class_names)}."
        )

    # Map predicted index to class label
    predicted_class = class_names[predicted_index]

    return predicted_class, confidence

In [36]:
class_names =['Aphids', 'Army worm', 'Bacterial blight', 'Cotton Boll Rot','Green Cotton Boll' ,
        'Healthy', 'Powdery mildew', 'Target spot' ]

# Example: Predict for a sample image
image_path = "/Users/shubhampund9767/Documents/Cotton-Disease-Recognition-using-YOLO-Algorithm--1/cotton.jpeg"
predicted_class, confidence = predict_image(image_path, class_names)

print(f"Predicted Class: {predicted_class}")
print(f"Confidence Score: {confidence:.2f}")

1/1 [==============================] - 0s 36ms/step
Raw predictions: [[9.6944696e-01 1.1075893e-02 3.3254514e-03 2.0897659e-04 1.3207029e-04
  2.9823869e-03 7.5654555e-03 5.2627702e-03]]
Predicted Class: Aphids
Confidence Score: 0.97
